In [1]:
import pandas as pd
import numpy as np

# Impact of Taxation and Inflation in Wealth Creation

The following notebook will show how to calculate the impact of personal taxation and inflation when calculating portfolio returns, whats the drag of taxes, how deferred taxes help extend long term compounding returns and how to minimze them as much as possible


### Tax structure for individual named A
#### She makes 700,000 USD on income, 10,000 on interest and 5,000 on dividends

In [6]:
# Constructing the DataFrame
tax_regime_dict = {
    'Taxable income over': [0, 30_000, 60_000, 90_000, 250_000, 500_000, 1_000_000],
    'Taxable income up to': [30_000, 60_000, 90_000, 250_000, 500_000, 1_000_000, None],
    'Tax': [None, 1_500, 4_500, 9_000, 41_000, 116_000, 316_000],
    'Percentage on excess over column 1 (%)': [0.05, 0.1, 0.15, 0.20, 0.30, 0.40, 0.50]
}

tax_regime = pd.DataFrame(tax_regime_dict)
tax_regime

,Taxable income over,Taxable income up to,Tax,Percentage on excess over column 1 (%)
0,0,30000.0,NaN,0.05
1,30000,60000.0,1500.0,0.10
2,60000,90000.0,4500.0,0.15
3,90000,250000.0,9000.0,0.20
4,250000,500000.0,41000.0,0.30
5,500000,1000000.0,116000.0,0.40
6,1000000,NaN,316000.0,0.50


In [18]:
ordinary_income = 700_000
interest_income = 10_000
dividend_income = 5_000
total_income = ordinary_income + interest_income + dividend_income
print(f'Total income for A is {total_income}')


tax_regime['Total income'] = total_income

def income_in_bracket(row,remaining_income,bracket_min:str='',bracket_max:str=''):
    # Determine the amount of income in this bracket 
    bracket_range = row[bracket_max] - row[bracket_min]
    taxable = min(bracket_range, max(0,remaining_income - row[bracket_min]))

#Sort by bracket_min to ensure order

tax_regime = tax_regime.sort_values(by='Taxable income over').reset_index(drop=True)

tax_regime['Tax Range'] = tax_regime['Taxable income up to'] - tax_regime['Taxable income over']

tax_regime.loc[0,'Adjusted income'] = tax_regime.loc[0,'Total income']

for i in range(1,len(tax_regime)):
    tax_regime.loc[i,'Adjusted income'] = tax_regime.loc[i-1,'Adjusted income'] - tax_regime.loc[i-1,'Tax Range']

tax_regime[]

Total income for A is 715000


,Taxable income over,Taxable income up to,Tax,Percentage on excess over column 1 (%),Total income,Tax Range,Adjusted income
0,0,30000.0,NaN,0.05,715000,30000.0,715000.0
1,30000,60000.0,1500.0,0.10,715000,30000.0,685000.0
2,60000,90000.0,4500.0,0.15,715000,30000.0,655000.0
3,90000,250000.0,9000.0,0.20,715000,160000.0,625000.0
4,250000,500000.0,41000.0,0.30,715000,250000.0,465000.0
5,500000,1000000.0,116000.0,0.40,715000,500000.0,215000.0
6,1000000,NaN,316000.0,0.50,715000,NaN,-285000.0


### Calculating tax drag in returns

In [39]:
tc = 0.2
rate = 0.07
n = 20
initial = 100_000

### Expected wealth at the end of 20 years
after_tax = np.round(initial*(1+rate)**20,2)

post_tax = np.round(initial*(1+rate*(1-tc))**20,2)

print(f'Final investment after tax is {after_tax} and post tax is {post_tax}')

tax_diff = after_tax - post_tax
print(f'Tax difference is {tax_diff} which is {tax_diff/(after_tax-initial)}')



Final investment after tax is 386968.45 and post tax is 297357.14
Tax difference is 89611.31 which is 0.31226885743014604
